# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']


J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 69 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.19113


extracting tarball to tmp_2503.19113...

 done.


Found 65 bibliographic references in tmp_2503.19113/OmegaCen_binaries.bbl.
Retrieving document from  https://arxiv.org/e-print/2503.19512


extracting tarball to tmp_2503.19512...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.19796


extracting tarball to tmp_2503.19796...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.19113-b31b1b.svg)](https://arxiv.org/abs/2503.19113) | **Studying Binary Systems in Omega Centauri with MUSE: II. Observational constraints on the orbital period distribution**  |
|| S. Saracino, et al. -- incl., <mark>J. Müller-Horn</mark>, <mark>N. Neumayer</mark>, <mark>M. Häberle</mark> |
|*Appeared on*| *2025-03-26*|
|*Comments*| *22 pages (Appendix A and B included), 16 Figures, 4 Tables. Accepted for publication in Monthly Notices of the Royal Astronomical Society*|
|**Abstract**|            Omega Centauri ($\omega$ Cen) is one of the most complex star clusters in the Milky Way, and likely the stripped nucleus of an accreted dwarf galaxy. Being the subject of debate between it hosting an intermediate-mass black hole (IMBH) or a collection of stellar-mass black holes (BHs) in its center, $\omega$ Cen has been intensively studied over the past decades. Our work focuses on characterizing the properties of binary systems in $\omega$ Cen via multi-epoch MUSE spectroscopic observations spanning over eight years and covering much of its central regions (i.e. core radius). We did not detect any stellar-mass BHs candidates orbiting luminous stars, although mock samples indicate a high sensitivity of our survey to such systems. This suggests that BHs orbiting stars may be rare in $\omega$ Cen or in wide orbits around low-mass companions (where our survey is 50% complete) or that the periods of such systems are longer than expected from cluster dynamics. Additionally, we constrained the orbital properties of 19 binary systems in the cluster, with periods ranging from fractions of a day up to several hundred days. We observe an excess of binaries with P $\ge$ 10 d and find evidence that the intrinsic period distribution of binaries in $\omega$ Cen differs from those predicted by cluster evolutionary models.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.19512-b31b1b.svg)](https://arxiv.org/abs/2503.19512) | **Conditional Autoencoder for Generating BNS Waveforms with Tidal and Precession Effects**  |
|| M. Sun, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-03-26*|
|*Comments*| **|
|**Abstract**|            Gravitational waves from binary neutron star mergers provide insights into dense matter physics and strong-field gravity, but waveform modeling remains computationally challenging. We develop a deep generative model for gravitational waveforms from BNS mergers, covering the late inspiral, merger, and ringdown, incorporating precession and tidal effects. Using the conditional autoencoder, our model efficiently generates waveforms with high fidelity across a broad parameter space, including component masses $(m_1, m_2)$, spin components $(S_{1x}, S_{1y}, S_{1z}, S_{2x}, S_{2y}, S_{2z})$, and tidal deformability $(\Lambda_1, \Lambda_2)$. Trained on $3 \times 10^5$ waveforms from the IMRPhenomXP\_NRTidalv2 waveform model, it achieves an average overlap accuracy of 99.6\% on the test set. The model significantly accelerates waveform generation. For a single sample, it requires $0.12$ seconds (s), compared to $0.38$ s for IMRPhenomXP\_NRTidalv2 and $0.62$ s for IMRPhenomPv2\_NRTidal, making it approximately 3 to 5 times faster. When generating $10^3$ waveforms, the network completes the task in $0.86$ s, while traditional waveform approximation methods take over $46$--$53$ s. Our model achieves a total time of $7.48$ s to generate $10^4$ such waveforms, making it about 60 to 65 times faster than traditional waveform approximation methods. This speed advantage enables rapid parameter estimation and real-time gravitational wave searches. With higher precision, it will support low-latency detection and broader applications in multi-messenger astrophysics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.19796-b31b1b.svg)](https://arxiv.org/abs/2503.19796) | **The cross-correlation between soft X-rays and galaxies A new benchmark for galaxy evolution models**  |
|| J. Comparat, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-26*|
|*Comments*| *Accepted in A&A*|
|**Abstract**|            This article presents the construction and validation of complete stellar mass-selected, volume-limited galaxy samples using the Legacy Survey (data release 10) galaxy catalogs, covering $\sim16,800$ deg$^2$ of extra-galactic sky, and extending to redshift $z<0.35$. We measure the correlation function of these galaxies with tiny statistical uncertainties at the percent level and systematic uncertainties up to 5\%. A 4-parameter halo occupation distribution (HOD) model is fitted to retrieve the population of host halos, yielding results on the stellar to halo mass relation consistent with the current models of galaxy formation and evolution. Using these complete galaxy samples, we measure and analyze the cross-correlation (X-corr) between galaxies and all soft X-ray photons observed by SRG/eROSITA in the 0.5-2 keV band over $\sim13,000$ deg$^2$. The cross correlation measurements have unprecedented sub-percent statistical uncertainty and ~5-10\% systematic uncertainty. An extension to the halo model is introduced to interpret the X-corr, decomposing contributions from X-ray point sources, hot gas (CGM), satellites, and the 2-halo term. For low stellar mass thresholds ($\log M^*/M_{\odot}>$ 10, 10.25, 10.5), we find that the point source emission dominates the X-corr at small separation ($r<80$kpc). Then, in the range ($80<r<2$Mpc), the emission from large halos hosting satellite galaxies dominates. Finally, on scales beyond that considered here ($r>2$Mpc), the 2-halo term becomes dominant. Interestingly, there is no scale at which the CGM dominates. In the range ($20<r<200$kpc), the CGM contributes to more than 10\% of the signal. Progressively, with the minimum stellar mass increasing, the CGM emission increases. We constrain the $M_{500c}-L_X$ scaling relation slope, $1.629^{+0.091}_{-0.089}$, at the 5\% level using the samples with the lowest mass threshold.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.19113/./OmegaCen_Pe_plot_mod.png', 'tmp_2503.19113/./OmegaCen_PK_plot_mod.png', 'tmp_2503.19113/./1664295_new-cropped.png', 'tmp_2503.19113/./1665349_new-cropped.png', 'tmp_2503.19113/./1670391_new-cropped.png', 'tmp_2503.19113/./1720096_new-cropped.png', 'tmp_2503.19113/./1724184_new-cropped.png', 'tmp_2503.19113/./1757804_new-cropped.png']
copying  tmp_2503.19113/./OmegaCen_Pe_plot_mod.png to _build/html/
copying  tmp_2503.19113/./OmegaCen_PK_plot_mod.png to _build/html/
copying  tmp_2503.19113/./1664295_new-cropped.png to _build/html/
copying  tmp_2503.19113/./1665349_new-cropped.png to _build/html/
copying  tmp_2503.19113/./1670391_new-cropped.png to _build/html/
copying  tmp_2503.19113/./1720096_new-cropped.png to _build/html/
copying  tmp_2503.19113/./1724184_new-cropped.png to _build/html/
copying  tmp_2503.19113/./1757804_new-cropped.png to _build/html/
exported in  _build/html/2503.19113.md
    + _build/html/tmp_2503.19113/./OmegaCen_Pe_plot_mod.png
  

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Studying Binary Systems in Omega Centauri with MUSE: II. Observational constraints on the orbital period distribution

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.19113-b31b1b.svg)](https://arxiv.org/abs/2503.19113)<mark>Appeared on: 2025-03-26</mark> -  _22 pages (Appendix A and B included), 16 Figures, 4 Tables. Accepted for publication in Monthly Notices of the Royal Astronomical Society_

</div>
<div id="authors">

S. Saracino, et al. -- incl., <mark>J. Müller-Horn</mark>, <mark>N. Neumayer</mark>, <mark>M. Häberle</mark>

</div>
<div id="abstract">

**Abstract:** Omega Centauri ( $\omega$ Cen) is one of the most complex star clusters in the Milky Way, and likely the stripped nucleus of an accreted dwarf galaxy. Being the subject of debate between it hosting an intermediate-mass black hole (IMBH) or a collection of stellar-mass black holes (BHs) in its center, $\omega$ Cen has been intensively studied over the past decades. Our work focuses on characterizing the properties of binary systems in $\omega$ Cen via multi-epoch MUSE spectroscopic observations spanning over eight years and covering much of its central regions (i.e. core radius). We did not detect any stellar-mass BHs candidates orbiting luminous stars, although mock samples indicate a high sensitivity of our survey to such systems. This suggests that BHs orbiting stars may be rare in $\omega$ Cen or in wide orbits around low-mass companions (where our survey is 50 \% complete) or that the periods of such systems are longer than expected from cluster dynamics. Additionally, we constrained the orbital properties of 19 binary systems in the cluster, with periods ranging from fractions of a day up to several hundred days. We observe an excess of binaries with P $\geq$ 10 d and find evidence that the intrinsic period distribution of binaries in $\omega$ Cen differs from those predicted by cluster evolutionary models.

</div>

<div id="div_fig1">

<img src="tmp_2503.19113/./OmegaCen_Pe_plot_mod.png" alt="Fig2" width="100%"/>

**Figure 2. -** Eccentricity - Period plot of the well-constrained binaries in $\omega$ Cen. Binaries with unimodal and bimodal solutions in the posterior period sampling are shown as black and red dots, respectively. Cyan diamonds identify binaries constrained by both The Joker and Ultranest. The period distribution of the 19 binaries is shown in gray in logarithmic scale and spans the range between 1 and 500 days with multiple peaks. The green histogram shows the same period distribution, once corrected for the incompleteness derived in Section \ref{sec:testI}. The eccentricity distribution, on the other hand, varies only from 0 to 0.5, with a peak around 0.1/0.15, i.e. prefers low eccentricity orbits. The dashed cyan line defines the maximum expected eccentricity for a given period. Binaries with P$<$ 2 days are expected to have circular or close to circular orbits. The reported values are from Ultranest. The values from The Joker can be found in Table \ref{tab:constrained} in the Appendix. (*fig:Pe_plot*)

</div>
<div id="div_fig2">

<img src="tmp_2503.19113/./OmegaCen_PK_plot_mod.png" alt="Fig3" width="100%"/>

**Figure 3. -** Period - Peak-to-Peak RV variation ($\Delta V_{r}$) plot of the 19 well constrained binaries in $\omega$ Cen. The colour code is the same as in Figure \ref{fig:Pe_plot}, also shown in the bottom-left legend. Stars with large orbital periods and/or high peak-to-peak RV variability occupy the upper right region of the plot. The dashed orange line defines the locus where equal-mass binaries composed of two stars with mass $0.8M_{\odot}$ -- the maximum stellar mass expected in $\omega$ Cen given its absolute age -- are located when observed edge-on. Binaries with q<1 are on the left of the orange line, while binaries with q>1 are on the right. The 1D period and $\Delta V_{r}$ distributions of the 19 binaries are also shown in the figure, in gray in logarithmic scale, spanning a large range of values. As in Fig. \ref{fig:Pe_plot}, the green histogram refers to the distribution of periods, once corrected for the results of Section \ref{sec:testI}. The reported values are from Ultranest. The values from The Joker, are provided in Table \ref{tab:constrained} in the Appendix. (*fig:PK_plot*)

</div>
<div id="div_fig3">

<img src="tmp_2503.19113/./1664295_new-cropped.png" alt="Fig13.1" width="16%"/><img src="tmp_2503.19113/./1665349_new-cropped.png" alt="Fig13.2" width="16%"/><img src="tmp_2503.19113/./1670391_new-cropped.png" alt="Fig13.3" width="16%"/><img src="tmp_2503.19113/./1720096_new-cropped.png" alt="Fig13.4" width="16%"/><img src="tmp_2503.19113/./1724184_new-cropped.png" alt="Fig13.5" width="16%"/><img src="tmp_2503.19113/./1757804_new-cropped.png" alt="Fig13.6" width="16%"/>

**Figure 13. -** Constrained binaries in $\omega$ Cen sorted by Star ID. The upper panel of every plot shows the observed RV curve (black points), the best-fit median model and the $\pm$ 1 $\sigma$ models (green continues lines). The green shaded area is the allowed region by propagating the uncertainties on the parameters. The lower panel shows the same RVs phase folded with the period from the median model. The colour code is the same as in upper panel. Moreover, it also contains the residuals after subtracting this model from the data. The reduced $\chi^2$ of the best-fit median model is also mentioned. (*fig:plot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.19113"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

407  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

40  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
